In [ ]:
#!pip install py3dep

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import scipy
from dask.distributed import Client
#import py3dep
import geopandas as gpd
import rasterio as rio
import pystac
import pystac_client
import stackstac
import math
import shapely
import matplotlib.pyplot as plt
import os
import xarray as xr
from datetime import datetime
#import ulmo
from datetime import datetime
import sys
import contextily as ctx
import rioxarray as rxr
#from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib
import matplotlib.transforms as mtransforms
import logging
import ipyleaflet
import planetary_computer
import requests
from IPython.display import Image
sys.path.append('../../../sar_snowmelt_timing')
import s1_rtc_bs_utils
import dask_gateway
from glob import glob

In [2]:
cluster = dask_gateway.GatewayCluster(shutdown_on_close=False) # 
client = cluster.get_client()
cluster.adapt(minimum=2, maximum=40)
print(client.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.a98cbb439f1b4a47975725c8c3f808ca/status


In [3]:
# mask out problem pixels 

classes = [ # page 13 of https://esa-worldcover.s3.amazonaws.com/v100/2020/docs/WorldCover_PUM_V1.0.pdf, comment out lines you want to REMOVE
#    10, # treecover
    20, # shrubland
    30, # grassland
    40, # cropland
#    50, # built-up
    60, #bare / sparse vegetation
    70, # snow and ice
#    80, # permanent water bodies
    90, # herbaceous wetlands
    95, # mangroves
    100 # loss and lichen
]    

snow_classes = [ # https://zenodo.org/record/2626737#.ZDMMf3bMIQ8
#    0, #Little-to-no snow
    1, #Indeterminate due to clouds
    2, #Ephemeral snow
    3, #Seasonal snow
]

In [4]:
gj = 'shapefiles/millcreek.geojson'
bbox_gdf = gpd.read_file(gj)

ts_ds = s1_rtc_bs_utils.get_s1_rtc_stac_pc(bbox_gdf,start_time='2022-10-01',end_time='2023-06-15', polarization='all',resolution=20)
worldcover = s1_rtc_bs_utils.get_worldcover(ts_ds)
snow_mask = s1_rtc_bs_utils.get_snowmask(ts_ds)

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
/srv/conda/envs/notebook/lib/python3.11/site-packages/pystac_client/item_search.py:826: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


In [5]:
worldcover

<xarray.DataArray 'map' (y: 1551, x: 1822)>
array([[30, 30, 30, ..., 10, 10, 10],
       [30, 30, 30, ..., 10, 10, 10],
       [30, 30, 30, ..., 10, 10, 10],
       ...,
       [30, 30, 30, ..., 10, 10, 10],
       [30, 30, 30, ..., 10, 10, 10],
       [30, 30, 30, ..., 10, 10, 10]], dtype=uint8)
Coordinates:
  * x            (x) float64 1.458e+06 1.458e+06 ... 1.495e+06 1.495e+06
  * y            (y) float64 5.107e+06 5.107e+06 ... 5.076e+06 5.076e+06
    spatial_ref  int64 0
Attributes:
    _FillValue:  255

In [6]:
snow_mask

<xarray.DataArray (y: 1551, x: 1822)>
array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
Coordinates:
  * x            (x) float64 1.458e+06 1.458e+06 ... 1.495e+06 1.495e+06
  * y            (y) float64 5.107e+06 5.107e+06 ... 5.076e+06 5.076e+06
    band         int64 1
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     255

In [8]:
from pathlib import Path


In [9]:
fn = Path(Path(os.path.abspath(s1_rtc_bs_utils.__file__)).parent.parent, 'input/SnowClass/westernUS_MODIS_snow_classes_byte.tif').as_posix()
    
snow_mask = rxr.open_rasterio(fn)

In [10]:
snow_mask

<xarray.DataArray (band: 1, y: 3024, x: 4527)>
[13689648 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -129.1 -129.1 -129.1 -129.1 ... -91.42 -91.41 -91.4
  * y            (y) float64 51.65 51.64 51.63 51.62 ... 26.48 26.47 26.46 26.45
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

2023-07-12 19:54:34,683 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,4))
snow_mask.plot(ax=ax[0],vmin=0,vmax=3)
worldcover.plot(ax=ax[1])

In [ ]:
mask_nosnow = True # true means remove areas without seasonal snow.
mask_vegetation = False # true means remove areas with trees

for pol in ['vv','vh','all']:
    print(f'Working on {pol}...')
    
    ts_ds = s1_rtc_bs_utils.get_s1_rtc_stac_pc(bbox_gdf,start_time='2022-10-01',end_time='2023-06-15', polarization=pol,resolution=20)
    
    if mask_nosnow:
        ts_ds = ts_ds.where(snow_mask.isin(snow_classes))
    if mask_vegetation:
        ts_ds = ts_ds.where(worldcover.isin(classes))
    
    melt_year = slice(f'2023-01-01',f'2023-06-15')
    ts_ds_oneyear = ts_ds.sel(time=melt_year) 
    runoffs_median = s1_rtc_bs_utils.get_runoff_onset(ts_ds_oneyear,num_acquisitions_during_melt_season=6)
    runoffs_median_computed = runoffs_median.dt.dayofyear.where(runoffs_median.dt.dayofyear>0).rio.write_nodata(-32768,encoded=True)
    runoffs_median_computed.rio.to_raster(f'runoff_onset_{pol}_snowmask_{mask_nosnow}_vegetationmask_{mask_vegetation}.tif',driver="COG", dtype='int16')

In [ ]:
files = sorted(glob('*vegetationmask_True*.tif'))

f,ax=plt.subplots(1,3,figsize=(20,5))

for file,axs in zip(files,ax):
    title = file.split('runoff_onset_')[1].split('.tif')[0]
    x = rxr.open_rasterio(file,mask_and_scale=True)
    x.plot(ax=axs,vmin=0,vmax=150)
    axs.set_title(f'{title}')
    axs.set_aspect('equal')

In [ ]:
files = sorted(glob('*vegetationmask_False*.tif'))

f,ax=plt.subplots(1,3,figsize=(20,5))

for file,axs in zip(files,ax):
    title = file.split('runoff_onset_')[1].split('.tif')[0]
    x = rxr.open_rasterio(file,mask_and_scale=True)
    x.plot(ax=axs,vmin=0,vmax=150)
    axs.set_title(f'{title}')
    axs.set_aspect('equal')

In [ ]:
#f,ax=plt.subplots(2,2,figsize=(15,10))
#s1_rtc_bs_utils.get_py3dep_dem(ts_ds).plot(ax=ax[0,0])
#s1_rtc_bs_utils.get_py3dep_aspect(ts_ds).plot(ax=ax[0,1])
#s1_rtc_bs_utils.get_py3dep_slope(ts_ds).plot(ax=ax[1,0])
#s1_rtc_bs_utils.get_dah(ts_ds).plot(ax=ax[1,1])
#plt.savefig("context.png", dpi=150)

In [ ]:
runoff = s1_rtc_bs_utils.get_runoff_onset(ts_ds,return_seperate_orbits=False).dt.dayofyear

In [ ]:
runoff

In [ ]:
#runoff.rio.to_raster("runoff.tiff")

In [ ]:
polygon = 'inputs/watershed/watershed_bound.shp'
boundary = gpd.read_file(polygon)

In [ ]:
lines = 'inputs/watershed/contour_lines1.shp'
contours = gpd.read_file(lines)

In [ ]:
boundary = boundary.to_crs(32610)

In [ ]:
contours = contours.to_crs(32610)

In [ ]:
boundary.plot(color='none')

In [ ]:
contours.plot()

In [ ]:
contours_clip = gpd.clip(contours,boundary)
contours_clip.plot()

In [ ]:
f,ax=plt.subplots(figsize=(10,8))
#runoff.plot(ax=ax,vmin=123,vmax=274,cmap='ocean_r')
runoff.plot(ax=ax,vmin=91,vmax=274,cmap='ocean_r')
boundary.plot(ax=ax,color='none',linestyle='-',linewidth=2)
contours_clip.plot(ax=ax,color='black',linestyle='-',linewidth=1)
plt.axis('off')
plt.title('Date of Runoff Onset, Mill Creek Watershed', pad=5)
#plt.savefig("outputs/runoff2.png", dpi=150)

In [ ]:
names = ['Mill Creek']
geojsons = ['inputs/geo/millcreek.geojson']

In [ ]:
for nm, gj in zip(names,geojsons):
    bbox_gdf = gpd.read_file(gj)
    #ts_ds = s1_rtc_bs_utils.get_s1_rtc_stac_pc(bbox_gdf,start_time='2015-01-01',end_time='2022-01-01',polarization='vv', resolution=40)
    ts_ds = s1_rtc_bs_utils.get_s1_rtc_stac_pc(bbox_gdf,start_time='2021-10-01',end_time='2022-09-30',polarization='vv', resolution=40)
    #ts_ds = s1_rtc_bs_utils.get_s1_rtc_stac_pc(bbox_gdf,start_time='2021-01-01',end_time='2022-01-01',polarization='vv', resolution=40)
    #ts_ds = s1_rtc_bs_utils.get_s1_rtc_stac_pc(bbox_gdf,start_time='2020-10-01',end_time='2021-09-30',polarization='vv', resolution=40)
    ts_ds = ts_ds.compute()
    ts_ds = ts_ds.dropna('time',how='all')
    
    #f,ax=plt.subplots(figsize=(30,7))
    
    #s1_rtc_bs_utils.plot_sentinel1_acquisitons(ts_ds,ax=ax)
    #f.suptitle(f'{nm}')
    #plt.tight_layout()
    #plt.savefig("acquisitions", dpi=150)

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,7))
s1_rtc_bs_utils.get_runoff_onset(ts_ds).dt.dayofyear.plot(ax=ax[0],cmap='twilight')
s1_rtc_bs_utils.get_ripening_onset(ts_ds).dt.dayofyear.plot(ax=ax[1],cmap='twilight')

In [ ]:
#summer_ndvi_ds = s1_rtc_bs_utils.get_median_ndvi(ts_ds)
summer_ndvi_ds = s1_rtc_bs_utils.get_median_ndvi(ts_ds,start_time='2022-07-30',end_time='2022-09-09')

In [ ]:
f,ax=plt.subplots()
summer_ndvi_ds.plot(ax=ax)
#plt.savefig("ndvi.png", dpi=150)

In [ ]:
s1_rtc_bs_utils.plot_backscatter_ts_and_ndvi(ts_ds,summer_ndvi_ds)
#plt.savefig("veg.png", dpi=150)

In [ ]:
resample_freq_s1='2W'
millcreek_sar_2w_ds = ts_ds.where(summer_ndvi_ds.values<0.2).resample(time='2W').mean(dim='time')
millcreek_sar_2w_ds.rio.write_crs(ts_ds.crs)
dem = s1_rtc_bs_utils.get_py3dep_dem(ts_ds)

In [ ]:
#f,ax=plt.subplots(4,2,figsize=(24,18),gridspec_kw={'width_ratios': [1, 8]})
f,ax=plt.subplots(2,2,figsize=(24,18),gridspec_kw={'width_ratios': [1, 8]})
s1_rtc_bs_utils.plot_timeseries_by_elevation_bin(ts_ds,dem,bin_size=100,ax=ax[0,1],normalize_bins=False)
s1_rtc_bs_utils.plot_timeseries_by_elevation_bin(ts_ds,dem,bin_size=100,ax=ax[1,1],normalize_bins=True)
s1_rtc_bs_utils.plot_hyposometry(ts_ds,dem,bin_size=100,ax=ax[0,0])
s1_rtc_bs_utils.plot_hyposometry(ts_ds,dem,bin_size=100,ax=ax[1,0])
#s1_rtc_bs_utils.plot_timeseries_by_elevation_bin(millcreek_sar_2w_ds,dem,bin_size=100,ax=ax[2,1],normalize_bins=False)
#s1_rtc_bs_utils.plot_timeseries_by_elevation_bin(millcreek_sar_2w_ds,dem,bin_size=100,ax=ax[3,1],normalize_bins=True)
#s1_rtc_bs_utils.plot_hyposometry(millcreek_sar_2w_ds,dem.where(summer_ndvi_ds.values<0.2),bin_size=100,ax=ax[2,0])
#s1_rtc_bs_utils.plot_hyposometry(millcreek_sar_2w_ds,dem.where(summer_ndvi_ds.values<0.2),bin_size=100,ax=ax[3,0])
ax[0,0].invert_xaxis()
ax[1,0].invert_xaxis()
#ax[2,0].invert_xaxis()
#ax[3,0].invert_xaxis()
#plt.tight_layout()
f.tight_layout(rect=[0, 0.03, 1, 0.95])
f.suptitle("Sentinel-1 SAR Backscatter binned by Elevation")
#plt.savefig("elevation", dpi=150)